In [406]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [407]:
v = 4

In [408]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.add_probs'
c_model = 'deberta-v3-small.flag-context4-2-d-oof.add_probs'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [409]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [410]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval.neg2/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

In [411]:
# d = pd.DataFrame(gezi.batch2list(xc))
# dg = d.groupby('pred_id')
# max_preds = dg['pred'].max()
# min_preds = dg['pred'].min()
# max_cls_preds = dg['cls_pred'].max()
# min_cls_preds = dg['cls_pred'].min()

In [412]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')
# xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')
xc = gezi.load(f'{root}/{c_model}.eval.160_30/valid.pkl')
xpc = gezi.load(f'{root}/{pc_model}.eval.neg2/valid.pkl')
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')

ic(calc_metric(xp, 'cls_pred'))
ic(calc_metric(xc, 'reg_pred'))
ic(calc_metric(xc, 'cls_pred'))
ic(calc_metric(xpc, 'cls_pred'))

[08/09/22 11:40:03] 1715974932.py:9 in <module>
                    calc_metric(xp, 'cls_pred'): 0.8940525259130645
[08/09/22 11:40:10] 1715974932.py:10 in <module>
                    calc_metric(xc, 'reg_pred'): 0.910446784805548
[08/09/22 11:40:19] 1715974932.py:11 in <module>
                    calc_metric(xc, 'cls_pred'): 0.9121324180804999
[08/09/22 11:40:27] 1715974932.py:12 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9011719210628208


0.9011719210628208

In [413]:
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  if prob.max() < 10:
  # if prob.max() < 0.8:
    marks.append(1)
    preds.append(pc_preds.get(cid, pred))
    probs.append(pc_probs.get(cid, prob))
    cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  else:
    marks.append(0)
    preds.append(pred)
    probs.append(prob)
    cls_preds.append(cls_pred)
    
xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())
ic(calc_metric(xpc, 'cls_pred'))

[08/09/22 11:40:38] 1343726780.py:27 in <module>
                    xpc['mark'].mean(): 1.0
[08/09/22 11:40:46] 1343726780.py:28 in <module>
                    calc_metric(xpc, 'cls_pred'): 0.9011719210628208


0.9011719210628208

In [414]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=10):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  # x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  # x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
        # x[f'top_rank_{i}'] = idxes[i]
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  # diff_feats(x, 'cls_rank_pred', 'rank_pred', 'cls_rank')
  return x

def gen_cfeat(x_):
  topn = 4
  x = {}
  keys = [
    # 'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    ]
  x = {k: x_[k] for k in keys}
  # x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  # x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  # x['reg_rank_pred'] = x['reg_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
        # x[f'tpop_rank_{i}']  = idxes[i] 
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      # x[f'top_rank_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  # diff_feats(x, 'cls_rank_pred', 'reg_rank_pred', 'cls_reg_rank')
  # diff_feats(x, 'cls_rank_pred', 'cls2_rank_pred', 'cls_cls2_rank')
  # diff_feats(x, 'cls2_rank_pred', 'reg_rank_pred', 'cls2_reg_rank')
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]

  feats = []
  for i in tqdm(range(len(xs)), desc='merge_feats'):
    fe1 = p_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')
    
    # diff_feats(fe, fe1['rank_pred'], fec['reg_rank_pred'], 'pc_cls_rank')
    # diff_feats(fe, fe1['cls_rank_pred'], fec['reg_rank_pred'], 'pc_cls_reg_rank')
    # diff_feats(fe, fepc['rank_pred'], fec['reg_rank_pred'], 'pcc_cls_rank')
    # diff_feats(fe, fepc['cls_rank_pred'], fec['reg_rank_pred'], 'pcc_cls_reg_rank')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)

    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [415]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [416]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [417]:
gezi.set_fold(df, 5, 'ancestor_id')

In [418]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

123

In [419]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [420]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [421]:
from catboost import CatBoostRegressor, CatBoostRanker, CatBoostClassifier
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              # 'reg_lambda': 7.960622217848342e-07, 
              'l2_leaf_reg': 3,
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              # 'max_depth': 10, 
              # 'max_depth': 8,
              'max_depth': 6,
              # 'max_depth': 5,
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [422]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
ic(cbt_score)

0:	learn: 0.2523098	test: 0.2523098	test1: 0.2522503	best: 0.2522503 (0)	total: 55.5ms	remaining: 9m 14s
100:	learn: 0.0661693	test: 0.0661693	test1: 0.0672994	best: 0.0672994 (100)	total: 4.13s	remaining: 6m 45s
200:	learn: 0.0594230	test: 0.0594230	test1: 0.0607075	best: 0.0607075 (200)	total: 7.91s	remaining: 6m 25s
300:	learn: 0.0586510	test: 0.0586510	test1: 0.0599722	best: 0.0599722 (300)	total: 11.8s	remaining: 6m 19s
400:	learn: 0.0578702	test: 0.0578702	test1: 0.0591987	best: 0.0591987 (400)	total: 15.9s	remaining: 6m 21s
500:	learn: 0.0573422	test: 0.0573422	test1: 0.0586798	best: 0.0586798 (500)	total: 19.9s	remaining: 6m 17s
600:	learn: 0.0569954	test: 0.0569954	test1: 0.0583523	best: 0.0583523 (600)	total: 24s	remaining: 6m 15s
700:	learn: 0.0567477	test: 0.0567477	test1: 0.0581247	best: 0.0581247 (700)	total: 28s	remaining: 6m 11s
800:	learn: 0.0565656	test: 0.0565656	test1: 0.0579680	best: 0.0579680 (800)	total: 31.9s	remaining: 6m 6s
900:	learn: 0.0564104	test: 0.056410

[08/09/22 11:49:16] 3198672829.py:8 in <module>
                    cbt_score: 0.9195142930981944


0.9195142930981944

In [423]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [426]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          # 'num_leaves': 32,
          # 'num_leaves': 24,
          'num_leaves': 12,
          'min_data_in_leaf': 300,
          # 'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                20000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=500)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.053480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29892
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 122
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 500 rounds
[100]	training's l1: 0.0677139	valid_1's l1: 0.0689357
[200]	training's l1: 0.0604585	valid_1's l1: 0.0618413
[300]	training's l1: 0.0594787	valid_1's l1: 0.0608751
[400]	training's l1: 0.0585858	valid_1's l1: 0.0600028
[500]	training's l1: 0.0577911	valid_1's l1: 0.0591996
[600]	training's l1: 0.0572957	valid_1's l1: 0.0587021
[700]	training's l1: 0.0569704	valid_1's l1: 0.0583885
[800]	training's l1: 0.0567878	valid_1's l1: 0.0582225
[900]	training's l1: 0.0566173	valid_1's l1: 0.0580742
[1000]	training's l1: 0.0564834	valid_1's l1: 0.0579707
[1100]	training's l1: 0.0564015	valid_1

[08/09/22 11:54:54] 3305026406.py:37 in <module>
                    lgb_score: 0.9195511977487746


0.9195511977487746

In [427]:
# model = lgb.LGBMRanker(
#     objective="lambdarank",
#     metric="ndcg",
#     boosting_type="dart",
#     n_estimators=100,
#     importance_type='gain',
#     verbose=10,
#     random_state = 17
# )

# qids_train = dtrain.groupby(['id'])['cell_id'].count().values
# # y_train2 = [int(x * 1000) for x in y_train]

# model.fit(
#     X=X_train,
#     y=y_train2,
#     group=qids_train,
# )

In [428]:
# import xgboost as xgb
# params = {
#       'reg_lambda': 7.960622217848342e-07, 
#                       'subsample': 0.7422597612762745,
#                       # 'bagging_temperature': 0.2,
#                       'max_depth': 10, 
#                       # 'early_stopping_rounds': 500,
#                       # 'n_estimators': 10000,
#                       # 'cat_features': [],
#                       # 'loss_function': 'MAE',
#                       'min_child_samples': 100,
#                       # eval_metric='MAE',
# }
# model = xgb.XGBRanker(booster="gbtree", 
#                       # objective="rank:ndcg",
#                       objective='rank:pairwise',
#                       tree_method="gpu_hist", 
#                       sampling_method="gradient_based",
#                       n_estimators=5000,
#                       learning_rate=0.03,
#                       **params
#                       )
# train_groups = dtrain.groupby('id').size().to_numpy()
# eval_groups = dvalid.groupby('id').size().to_numpy()
# model.fit(X_train, 
#           y_train, 
#           group=train_groups, 
#           eval_set=[(X_train, y_train),(X_valid, y_valid)],
#           eval_group=[train_groups, eval_groups],
#           verbose=100)


In [429]:
# xgb_model = model

In [430]:
# dvalid['xgb_pred'] = xgb_model.predict(dvalid[cols])
# dvalid['xgb_pred'] = [gezi.sigmoid(x) for x in dvalid.xgb_pred.values]
# xgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.xgb_pred.values})
# ic(xgb_score)

In [431]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [432]:
dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
cbt_score

0.9195142930981944

In [433]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir(f'../working/trees{v}')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees{v}/{fold}.cbt')
  dvalid['cbt_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cbt_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    10000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=100)
  lgb_model.save_model(f'../working/trees{v}/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cbt_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/09/22 11:59:25] 4217963489.py:21 in <module>
                    cbt_score: 0.9195142930981944
[08/09/22 12:00:06] 4217963489.py:36 in <module>
                    lgb_score: 0.9195511977487746
[08/09/22 12:00:13] 4217963489.py:38 in <module>
                    score: 0.9198101558053882
[08/09/22 12:00:13] 4217963489.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9195142930981944
                    np.asarray(lgb_scores).mean(): 0.9195511977487746
                    np.asarray(scores).mean(): 0.9198101558053882


0:	learn: 0.2523304	test: 0.2523304	test1: 0.2522116	best: 0.2522116 (0)	total: 54.3ms	remaining: 9m 3s
500:	learn: 0.0577707	test: 0.0577707	test1: 0.0569862	best: 0.0569862 (500)	total: 20.8s	remaining: 6m 33s
1000:	learn: 0.0566185	test: 0.0566185	test1: 0.0560441	best: 0.0560441 (1000)	total: 49.4s	remaining: 7m 24s
1500:	learn: 0.0561655	test: 0.0561655	test1: 0.0557697	best: 0.0557697 (1500)	total: 1m 18s	remaining: 7m 24s
2000:	learn: 0.0557905	test: 0.0557905	test1: 0.0556044	best: 0.0556044 (2000)	total: 1m 46s	remaining: 7m 7s
2500:	learn: 0.0554716	test: 0.0554716	test1: 0.0554951	best: 0.0554949 (2498)	total: 2m 26s	remaining: 7m 17s
3000:	learn: 0.0552276	test: 0.0552276	test1: 0.0554209	best: 0.0554208 (2999)	total: 3m 8s	remaining: 7m 18s
3500:	learn: 0.0549877	test: 0.0549877	test1: 0.0553676	best: 0.0553676 (3500)	total: 3m 59s	remaining: 7m 24s
4000:	learn: 0.0547483	test: 0.0547483	test1: 0.0553201	best: 0.0553201 (3999)	total: 4m 49s	remaining: 7m 13s
4500:	learn: 0

[08/09/22 12:11:35] 4217963489.py:21 in <module>
                    cbt_score: 0.9218303226369757


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.067247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29892
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 122
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.058338	valid_1's l1: 0.0574237
[1000]	training's l1: 0.057003	valid_1's l1: 0.0562923
[1500]	training's l1: 0.0565813	valid_1's l1: 0.0559892
[2000]	training's l1: 0.0563491	valid_1's l1: 0.0558518
[2500]	training's l1: 0.0561449	valid_1's l1: 0.0557391
[3000]	training's l1: 0.0560164	valid_1's l1: 0.0556815
[3500]	training's l1: 0.0558988	valid_1's l1: 0.0556258
[4000]	training's l1: 0.0557678	valid_1's l1: 0.0555662
[4500]	training's l1: 0.0556232	valid_1's l1: 0.0555128
[5000]	training's l1: 0.055534	valid_1's l1: 0.0554745
[5500]	training's l1: 0.0554652	va

...

[08/09/22 12:43:12] 4217963489.py:36 in <module>
                    lgb_score: 0.9242246875276904
[08/09/22 12:43:14] 4217963489.py:38 in <module>
                    score: 0.924527443211856
[08/09/22 12:43:14] 4217963489.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.921764285556478
                    np.asarray(lgb_scores).mean(): 0.92156622185366
                    np.asarray(scores).mean(): 0.9218647262670266


0:	learn: 0.2523110	test: 0.2523110	test1: 0.2523947	best: 0.2523947 (0)	total: 65.6ms	remaining: 10m 55s
500:	learn: 0.0570652	test: 0.0570652	test1: 0.0585926	best: 0.0585926 (500)	total: 21.4s	remaining: 6m 45s
1000:	learn: 0.0561366	test: 0.0561366	test1: 0.0578004	best: 0.0578004 (1000)	total: 42.9s	remaining: 6m 25s
1500:	learn: 0.0556768	test: 0.0556768	test1: 0.0575164	best: 0.0575164 (1500)	total: 1m 3s	remaining: 6m 2s
2000:	learn: 0.0553477	test: 0.0553477	test1: 0.0573626	best: 0.0573626 (2000)	total: 1m 25s	remaining: 5m 40s
2500:	learn: 0.0550654	test: 0.0550654	test1: 0.0572681	best: 0.0572681 (2500)	total: 1m 45s	remaining: 5m 17s
3000:	learn: 0.0547958	test: 0.0547958	test1: 0.0571872	best: 0.0571872 (3000)	total: 2m 6s	remaining: 4m 55s
3500:	learn: 0.0545641	test: 0.0545641	test1: 0.0571456	best: 0.0571446 (3461)	total: 2m 28s	remaining: 4m 34s
4000:	learn: 0.0543523	test: 0.0543523	test1: 0.0571049	best: 0.0571049 (4000)	total: 2m 48s	remaining: 4m 12s
4500:	learn: 

[08/09/22 12:54:32] 4217963489.py:21 in <module>
                    cbt_score: 0.9150272742672104


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.113429 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29891
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 122
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.0576506	valid_1's l1: 0.0590757
[1000]	training's l1: 0.0564383	valid_1's l1: 0.0580329
[1500]	training's l1: 0.0560885	valid_1's l1: 0.057773
[2000]	training's l1: 0.0558579	valid_1's l1: 0.0576227
[2500]	training's l1: 0.0556868	valid_1's l1: 0.0575338
[3000]	training's l1: 0.0555762	valid_1's l1: 0.0574783
[3500]	training's l1: 0.0554804	valid_1's l1: 0.0574247
[4000]	training's l1: 0.0553867	valid_1's l1: 0.0573754
[4500]	training's l1: 0.0552903	valid_1's l1: 0.0573412
[5000]	training's l1: 0.0552184	valid_1's l1: 0.0573108
[5500]	training's l1: 0.055164	v

[08/09/22 12:57:50] 4217963489.py:36 in <module>
                    lgb_score: 0.9145454838112385
[08/09/22 12:57:52] 4217963489.py:38 in <module>
                    score: 0.9149569766597708
[08/09/22 12:57:53] 4217963489.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9204168832986245
                    np.asarray(lgb_scores).mean(): 0.9201620742451757
                    np.asarray(scores).mean(): 0.9204831763455754


In [434]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614